## import modules

In [3]:
import pandas as pd

## pre-process census data

In [4]:
# for population data
cols=['State', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F']
census=pd.read_excel('datasets/census.xlsx',engine='openpyxl',usecols=cols)
censusIndia=census.iloc[0,:]
census=census.loc[census.Level=='STATE']
census=census.loc[census.TRU=='Total']
census=census.append(censusIndia,ignore_index=True)
census.sort_values(by=['State'],axis=0,inplace=True)
census.reset_index(drop=True,inplace=True)

## define function 
- that finds ratios for state/ut given its code

- particular details are given in comments

In [5]:
#===================
# # a function that just takes state code that does
# manipulates and finds appropriate sum
# finds total pop fro that state from census data
# then returns a nice ratio dictionary
#===================

def ratioFinder(stateCode):
    # 5 is specific to TOT_P
    statePop=census.iloc[stateCode,5] 
    # modify state code
    stateCode_str=str(stateCode).zfill(2)
    # read df from file
    df=pd.read_excel(f'datasets/C-17/{stateCode_str}.xlsx',skiprows=6,header=None,engine='openpyxl')
    df.fillna(value=0,inplace=True)
    # find total no 
    # note: these col numbers are specific to df #
    total3=sum(df.iloc[:,14].tolist())
    total2=sum(df.iloc[:,9].tolist())-total3 # need to remove total3 as itself is of 2+ language(that also includes 3+ lang persons) speakers
    total1=sum(df.iloc[:,4].tolist())-total2-total3 # as previously I was double substracting 3+ persons I was getting negative number for that
    # doing -toatal3 because total2 has been modified above with toatl3 being substracted
    ratio_1=round(100*total1/statePop,2)
    ratio_2=round(100*total2/statePop,2)
    ratio_3=round(100*total3/statePop,2)
    #store ratios in a dict
    stateRatios={
        'state-code':stateCode,
        'percent-one':ratio_1,
        'percent-two':ratio_2,
        'percent-three':ratio_3,
    }
    #print('Processed state -- ',stateCode)
    return stateRatios


## find ratios for states 
- using above func and store them into a csv file

In [6]:
%%time
ratioList=[ratioFinder(i) for i in range(0,36)]
ratioDF=pd.DataFrame(ratioList)
ratioDF.to_csv('outputs/percent-india.csv',index=False)

CPU times: user 19.4 s, sys: 44.6 ms, total: 19.4 s
Wall time: 19.5 s


In [ ]:
ratioDF

## observations

- for most of the state/ut majority population speaks only one language. Exactly two languages speaking population is lower than only one language speaking population. And 3+ languages speaking population is even less. For some exception state/ut it is not true, but these are only one or two states.
- These findings are intutive; we expect minor proportion of popultaion to speak more than 3_ languages and major proportion of popultaion to speak only one language.
- As expected sum of all three ratios is 100%; meaning it covers whole population of state/ut   